# Homework 2: SQL in Python (Data and Databases 2022)

The next four homework assignments will be in this iPython notebook format. Read the instructions as you go, and type in the necessary code--then run the cells to see if you got it right. Don't worry if you are new to python for this assignment--the only "code" you will be writing are SQL statements that you will paste into each myquery variable.


## The Challenges

For this homework we will be using the tables we imported from the [UN data site](http://data.un.org/Explorer.aspx). You should already have `wind` and `solar` installed. As you will see, this homework asks that you find one more table of your choice from the UN's Energy statistics an import it into the the same database. [I have posted the instructions here](http://floatingmedia.com/columbia/csv_to_sql.html)

These are relatively simple CSV files--the only really important columns are `country`, `year` and `usage`. But the tricky part is that most countries have multiple entries: one for every year the UN has been gathering data on that country. So most of these queries are going to ask you to get electrical usage results based on country and year. (Major hint: JOINs are going to need to be based on country AND year.)

In the questions below, we focus on exploratory queries--with an emphasis on aggregates and joins. Read the questions carefully, and then write SQL statements that will get you the correct results. (You will want to explore tables and test out queries directly in the shell before you paste them into Python--the error handling for SQL in Python can be confusing, and there should be no errors in the prewritten Python parts of this document. Also, sometimes an error in Python will break your connection with the database--see conn.rollback() below for that solution.


First things first: Connect to database! 
(Make sure the cell is selected, then go to the cell menubar and choose "Run Cells")

In [167]:
import pg8000
# use whatever username information you used in classed to connect
conn = pg8000.connect(database="un_data",user="laurabejderjensen")
print(type(conn))

<class 'pg8000.legacy.Connection'>


If the connection isn't working make sure that you have typed in the name of the database correctly, and your username as well. 

If you get an error when running one of the SQL cells below, run this conn.rollback() cell to restart your connection.

In [168]:
conn.rollback()

## Question 1: Simple(ish) Query 

Create a statement that searches for the top 15 countries for solar energy usage in 2014. So figure out first what table you are searching and what columns you need (you can figure out the columns by looking at the sample results below). Then you need to test by year so you only get results from 2014, and order it correctly.

```
Germany 36056.0 mil
United States 25764.0 mil
China 25134.0 mil
Japan 22952.0 mil
Italy 22306.0 mil
Spain 13673.0 mil
France 6392.0 mil
India 5020.0 mil
United Kingdom 4054.0 mil
Australia 4010.0 mil
Greece 3792.0 mil
Belgium 2886.0 mil
Korea, Republic of 2557.0 mil
Czechia 2123.0 mil
Canada 2120.0 mil
```

In [169]:
cursor = conn.cursor()
myquery = ''' 
SELECT country, usage 
FROM solar 
WHERE year = 2014 
ORDER BY usage DESC NULLS LAST 
LIMIT 15;
'''
cursor.execute(myquery)
for row in cursor:
    print(row[0], row[1], "mil")

Germany 36056.0 mil
United States 25764.0 mil
China 25134.0 mil
Japan 22952.0 mil
Italy 22306.0 mil
Spain 13673.0 mil
France 6392.0 mil
India 5020.0 mil
United Kingdom 4054.0 mil
Australia 4010.0 mil
Greece 3792.0 mil
Belgium 2886.0 mil
Korea, Republic of 2557.0 mil
Czechia 2123.0 mil
Canada 2120.0 mil


## Question 2: aggregate query

Using avg(), get the average solar energy usage for each country, and display only the countries with an average that is above 500 (million). Make sure it is properly ordered like the results below, and wrap your avg column in round() to get round numbers:

```
91339.0 China
20425.0 India
14084.0 United States
12142.0 Germany
11094.0 Japan
6414.0 Italy
4488.0 Spain
3532.0 Turkey
2992.0 United Kingdom
2733.0 Brazil
2631.0 France
2215.0 Chile
2098.0 Australia
1875.0 Greece
1775.0 Thailand
1702.0 Belgium
1634.0 South Africa
1545.0 Korea, Republic of
1462.0 Czechia
1284.0 Jordan
1044.0 Other Asia
1002.0 Israel
955.0 Bulgaria
896.0 Honduras
865.0 Romania
859.0 Canada
855.0 United Arab Emirates
794.0 Ukraine
671.0 Morocco
600.0 Pakistan
586.0 Russian Federation
543.0 Netherlands
```

In [170]:
cursor = conn.cursor()
myquery = '''
SELECT country, round(avg(usage))
FROM solar
GROUP BY country
HAVING round(avg(usage)) > 500
ORDER BY round(avg(usage)) DESC;
'''
cursor.execute(myquery)
for row in cursor:
    print(row[0],row[1])

China 91339.0
India 20425.0
United States 14084.0
Germany 12142.0
Japan 11094.0
Italy 6414.0
Spain 4488.0
Turkey 3532.0
United Kingdom 2992.0
Brazil 2733.0
France 2631.0
Chile 2215.0
Australia 2098.0
Greece 1875.0
Thailand 1775.0
Belgium 1702.0
South Africa 1634.0
Korea, Republic of 1545.0
Czechia 1462.0
Jordan 1284.0
Other Asia 1044.0
Israel 1002.0
Bulgaria 955.0
Honduras 896.0
Romania 865.0
Canada 859.0
United Arab Emirates 855.0
Ukraine 794.0
Morocco 671.0
Pakistan 600.0
Russian Federation 586.0
Netherlands 543.0


## Question 3: aggregate query 2

Get a list of countries whose Solar Energy data first started being collected in 2012. This is tricky to figure out, but the query should be pretty simple. The main thing you need to figure out is which aggregate function will find the earliest date, and then you need to test if year is 2012. And make it alphabetical by country.


```
2012 Botswana
2012 Fiji
2012 Gabon
2012 Haiti
2012 Lao People's Dem. Rep.
2012 Lithuania
2012 Marshall Islands
2012 Mauritius
2012 Nigeria
2012 Poland
2012 South Sudan
2012 St. Helena and Depend.
2012 Sudan
2012 Suriname
2012 United Arab Emirates
```


In [171]:
cursor = conn.cursor()
myquery = '''
SELECT country, min(year)
FROM solar
GROUP BY country
HAVING min(year) = 2012
ORDER BY country ASC;
'''
cursor.execute(myquery)
for row in cursor:
    print(row[0], row[1])

Botswana 2012
Fiji 2012
Gabon 2012
Haiti 2012
Lao People's Dem. Rep. 2012
Lithuania 2012
Marshall Islands 2012
Mauritius 2012
Nigeria 2012
Poland 2012
South Sudan 2012
St. Helena and Depend. 2012
Sudan 2012
Suriname 2012
United Arab Emirates 2012


## Question 4: JOIN

Get the usage of wind AND solar power for Germany for each year in the tables. There should be four columns total, and you will be joining on two criteria.

(Please note that in the results below I am using Python to add in the strings "solar:" and "wind:")



```
Germany 1991 	wind: 215.0 	solar: 1.0
Germany 1992 	wind: 291.0 	solar: 4.0
Germany 1993 	wind: 674.0 	solar: 3.0
Germany 1994 	wind: 1428.0 	solar: 7.0
Germany 1995 	wind: 1712.0 	solar: 7.0
Germany 1996 	wind: 2078.0 	solar: 12.0
Germany 1997 	wind: 3034.0 	solar: 18.0
Germany 1998 	wind: 4593.0 	solar: 35.0
Germany 1999 	wind: 5528.0 	solar: 30.0
Germany 2000 	wind: 9352.0 	solar: 60.0
Germany 2001 	wind: 10456.0 	solar: 116.0
Germany 2002 	wind: 15856.0 	solar: 188.0
Germany 2003 	wind: 18713.0 	solar: 313.0
Germany 2004 	wind: 25509.0 	solar: 557.0
Germany 2005 	wind: 27229.0 	solar: 1282.0
Germany 2006 	wind: 30710.0 	solar: 2220.0
Germany 2007 	wind: 39713.0 	solar: 3075.0
Germany 2008 	wind: 40574.0 	solar: 4420.0
Germany 2009 	wind: 38647.0 	solar: 6584.0
Germany 2010 	wind: 38547.0 	solar: 11729.0
Germany 2011 	wind: 49858.0 	solar: 19599.0
Germany 2012 	wind: 51680.0 	solar: 26380.0
Germany 2013 	wind: 52737.0 	solar: 31010.0
Germany 2014 	wind: 58497.0 	solar: 36056.0
Germany 2015 	wind: 80624.0 	solar: 38726.0
Germany 2016 	wind: 79924.0 	solar: 38098.0
Germany 2017 	wind: 105693.0 	solar: 39401.0
Germany 2018 	wind: 109951.0 	solar: 45784.0
Germany 2019 	wind: 125894.0 	solar: 46392.0
```



In [172]:
cursor = conn.cursor()
myquery = '''
SELECT solar.country, solar.year, wind.usage, solar.usage
FROM solar
JOIN wind ON wind.country=solar.country AND wind.year=solar.year
WHERE solar.country = 'Germany'
ORDER BY year ASC;
'''
cursor.execute(myquery)
for row in cursor:
    print(row[0],row[1],"\twind:",row[2],"\tsolar:",row[3])

Germany 1991 	wind: 215.0 	solar: 1.0
Germany 1992 	wind: 291.0 	solar: 4.0
Germany 1993 	wind: 674.0 	solar: 3.0
Germany 1994 	wind: 1428.0 	solar: 7.0
Germany 1995 	wind: 1712.0 	solar: 7.0
Germany 1996 	wind: 2078.0 	solar: 12.0
Germany 1997 	wind: 3034.0 	solar: 18.0
Germany 1998 	wind: 4593.0 	solar: 35.0
Germany 1999 	wind: 5528.0 	solar: 30.0
Germany 2000 	wind: 9352.0 	solar: 60.0
Germany 2001 	wind: 10456.0 	solar: 116.0
Germany 2002 	wind: 15856.0 	solar: 188.0
Germany 2003 	wind: 18713.0 	solar: 313.0
Germany 2004 	wind: 25509.0 	solar: 557.0
Germany 2005 	wind: 27229.0 	solar: 1282.0
Germany 2006 	wind: 30710.0 	solar: 2220.0
Germany 2007 	wind: 39713.0 	solar: 3075.0
Germany 2008 	wind: 40574.0 	solar: 4420.0
Germany 2009 	wind: 38647.0 	solar: 6584.0
Germany 2010 	wind: 38547.0 	solar: 11729.0
Germany 2011 	wind: 49858.0 	solar: 19599.0
Germany 2012 	wind: 51680.0 	solar: 26380.0
Germany 2013 	wind: 52737.0 	solar: 31010.0
Germany 2014 	wind: 58497.0 	solar: 36056.0
Germa

## Question 5: join with aggregates

Get the average wind usage and solar usage for each country. Order your results by average solar usage, and show only the top ten countries.

```
China wind: 184643.09 	solar: 91339.29
India wind: 14906.80 	solar: 20424.86
United States wind: 75203.79 	solar: 14083.68
Germany wind: 35507.48 	solar: 12141.62
Japan wind: 2446.25 	solar: 11093.53
Italy wind: 5955.78 	solar: 6414.15
Spain wind: 23194.07 	solar: 4488.50
Turkey wind: 5431.81 	solar: 3532.15
United Kingdom wind: 13128.51 	solar: 2991.66
Brazil wind: 20745.71 	solar: 2733.12
```

In [173]:
cursor = conn.cursor()
myquery = '''
SELECT solar.country, round(avg(wind.usage)) AS wind, round(avg(solar.usage)) AS solar
FROM solar
JOIN wind ON wind.country=solar.country
GROUP BY solar.country
ORDER BY avg(solar.usage) DESC
LIMIT 10;

'''
cursor.execute(myquery)
for row in cursor:
    print(row[0],"wind:","%.2f" % row[1],"\tsolar:","%.2f" % row[2])

China wind: 184643.00 	solar: 91339.00
India wind: 14907.00 	solar: 20425.00
United States wind: 75204.00 	solar: 14084.00
Germany wind: 35507.00 	solar: 12142.00
Japan wind: 2446.00 	solar: 11094.00
Italy wind: 5956.00 	solar: 6414.00
Spain wind: 23194.00 	solar: 4488.00
Turkey wind: 5432.00 	solar: 3532.00
United Kingdom wind: 13129.00 	solar: 2992.00
Brazil wind: 20746.00 	solar: 2733.00


## Question 6: same thing but slightly more complicated

Get the average wind usage and solar usage for each country, FROM 2010 TO THE PRESENT. Order your results by average solar usage, and show only the top ten countries.

```
China wind: 200417.40 	solar: 91339.29
United States wind: 196527.57 	solar: 40154.94
Germany wind: 75340.50 	solar: 33317.50
Japan wind: 5732.55 	solar: 31807.88
India wind: 36939.70 	solar: 20424.86
Italy wind: 15065.74 	solar: 19122.54
Spain wind: 49822.00 	solar: 12506.80
France wind: 20167.67 	solar: 6808.27
United Kingdom wind: 35476.83 	solar: 6274.08
Australia wind: 10545.80 	solar: 5567.01
```

In [174]:
cursor = conn.cursor()
myquery = '''
SELECT solar.country, round(avg(wind.usage)), round(avg(solar.usage))
FROM solar
JOIN wind ON wind.country=solar.country
WHERE solar.year > 2009 AND wind.year > 2009
GROUP BY solar.country
ORDER BY avg(solar.usage) DESC
LIMIT 10;

'''
cursor.execute(myquery)
for row in cursor:
    print(row[0],"wind:","%.2f" % row[1],"\tsolar:","%.2f" % row[2])

China wind: 200417.00 	solar: 91339.00
United States wind: 196528.00 	solar: 40155.00
Germany wind: 75340.00 	solar: 33318.00
Japan wind: 5733.00 	solar: 31808.00
India wind: 36940.00 	solar: 20425.00
Italy wind: 15066.00 	solar: 19123.00
Spain wind: 49822.00 	solar: 12507.00
France wind: 20168.00 	solar: 6808.00
United Kingdom wind: 35477.00 	solar: 6274.00
Australia wind: 10546.00 	solar: 5567.00


## Question 7: Building a basic timeseries

Step one: get a list of the first year of data and the most recent year of data for each country from the solar table. Limit the results to the first 15 with the oldest year and then in alphabetical order.

```
Italy 1990 -- 2019
Japan 1990 -- 2019
Korea, Republic of 1990 -- 2019
Mexico 1990 -- 2019
Portugal 1990 -- 2019
Spain 1990 -- 2019
Switzerland 1990 -- 2019
United States 1990 -- 2019
Finland 1991 -- 2019
Germany 1991 -- 2019
Netherlands 1991 -- 2019
Canada 1992 -- 2019
Australia 1993 -- 2019
Austria 1993 -- 2019
Liechtenstein 1993 -- 2020
```

In [175]:
cursor = conn.cursor()
myquery = '''
SELECT country, min(year), max(year)
FROM solar
GROUP BY country
ORDER BY min(year) ASC, country ASC
LIMIT 15;

'''
cursor.execute(myquery)
for row in cursor:
    print(row[0],row[1],"--",row[2])

Italy 1990 -- 2019
Japan 1990 -- 2019
Korea, Republic of 1990 -- 2019
Mexico 1990 -- 2019
Portugal 1990 -- 2019
Spain 1990 -- 2019
Switzerland 1990 -- 2019
United States 1990 -- 2019
Finland 1991 -- 2019
Germany 1991 -- 2019
Netherlands 1991 -- 2019
Canada 1992 -- 2019
Australia 1993 -- 2019
Austria 1993 -- 2019
Liechtenstein 1993 -- 2020


## Question 8: Building a basic timeseries

Step two: Take that previous query, (order, limit and everything) and turn that into a subquery, and join the solar table on it twice, first to get the usage from the earliest date, next get the usage from the most recent date. Order it by the most usage from the most recent year.

Note: to join at the same table twice, you need to alias the table (solar as sl1, solar as sl2). This is getting very challenging, so get as far as you can.
```
United States 1990 -- 2019 usage change:  666.0 -- 97478.316
Japan 1990 -- 2019 usage change:  67.0 -- 68952.571
Germany 1991 -- 2019 usage change:  1.0 -- 46392.0
Italy 1990 -- 2019 usage change:  4.0 -- 23688.9
Spain 1990 -- 2019 usage change:  9.0 -- 15103.0
Australia 1993 -- 2019 usage change:  11.0 -- 14848.0
Korea, Republic of 1990 -- 2019 usage change:  1.0 -- 12996.018
Mexico 1990 -- 2019 usage change:  1.0 -- 7057.0
Netherlands 1991 -- 2019 usage change:  1.0 -- 5335.455
Canada 1992 -- 2019 usage change:  2.0 -- 4079.0
Switzerland 1990 -- 2019 usage change:  1.0 -- 2177.671
Austria 1993 -- 2019 usage change:  1.0 -- 1702.093
Portugal 1990 -- 2019 usage change:  1.0 -- 1342.252
Finland 1991 -- 2019 usage change:  1.0 -- 147.25
Liechtenstein 1993 -- 2020 usage change:  0.008 -- 29.7
```

In [176]:
cursor = conn.cursor()
myquery = '''
WITH subtable AS (SELECT country, min(year) AS min_year, max(year) AS max_year
FROM solar
GROUP BY country
ORDER BY min_year ASC, country ASC
LIMIT 15)
SELECT subtable.country, subtable.min_year, subtable.max_year, sl1.usage, sl2.usage
FROM subtable
JOIN solar AS sl1
ON subtable.country = sl1.country AND subtable.min_year = sl1.year
JOIN solar AS sl2
ON subtable.country = sl2.country AND subtable.max_year = sl2.year
ORDER BY sl2.usage DESC NULLS LAST;
'''  
cursor.execute(myquery)
for row in cursor:
    print(row[0],row[1],"--",row[2],"usage change: ", row[3],"--",row[4])

United States 1990 -- 2019 usage change:  666.0 -- 97478.316
Japan 1990 -- 2019 usage change:  67.0 -- 68952.571
Germany 1991 -- 2019 usage change:  1.0 -- 46392.0
Italy 1990 -- 2019 usage change:  4.0 -- 23688.9
Spain 1990 -- 2019 usage change:  9.0 -- 15103.0
Australia 1993 -- 2019 usage change:  11.0 -- 14848.0
Korea, Republic of 1990 -- 2019 usage change:  1.0 -- 12996.018
Mexico 1990 -- 2019 usage change:  1.0 -- 7057.0
Netherlands 1991 -- 2019 usage change:  1.0 -- 5335.455
Canada 1992 -- 2019 usage change:  2.0 -- 4079.0
Switzerland 1990 -- 2019 usage change:  1.0 -- 2177.671
Austria 1993 -- 2019 usage change:  1.0 -- 1702.093
Portugal 1990 -- 2019 usage change:  1.0 -- 1342.252
Finland 1991 -- 2019 usage change:  1.0 -- 147.25
Liechtenstein 1993 -- 2020 usage change:  0.008 -- 29.7


## Question 9: join with aggregates, new data

Your final task: go back to the [UN data site](http://data.un.org/Explorer.aspx) and download another table of your choice from the energy statistics. [Full instructions here.](http://floatingmedia.com/columbia/csv_to_sql.html) Choose a table that has similar columns to the wind and energy tables, and input it into your database.  Make sure you test that everything is working in the shell before going back to Python.

Now, write a query that joins the third table to the query you made in Question 5. Get the average use for each Energy type: Solar, Wind, and the table you chose. Order it by the energy type you chose (that column should come first after the country). And display the top 10 countries by the average of the energy type that you added, along with the averages for solar and wind.


In [177]:
cursor = conn.cursor()
myquery = '''
SELECT solar.country, round(avg(wind.usage)) AS wind, round(avg(solar.usage)) AS solar, round(avg(nuclear.usage)) AS nuclear
FROM solar
INNER JOIN wind ON wind.country=solar.country
INNER JOIN nuclear ON nuclear.country=solar.country
GROUP BY solar.country
ORDER BY avg(solar.usage) DESC
LIMIT 10;
'''
cursor.execute(myquery)
for row in cursor:
    print(row[0],"%.2f" % row[1],"%.2f" % row[2],"%.2f" % row[3])

China 184643.00 91339.00 83073.00
India 14907.00 20425.00 20615.00
United States 75204.00 14084.00 775318.00
Germany 35507.00 12142.00 136672.00
Japan 2446.00 11094.00 206422.00
Spain 23194.00 4488.00 58134.00
United Kingdom 13129.00 2992.00 77001.00
Brazil 20746.00 2733.00 10182.00
France 8297.00 2631.00 407489.00
Belgium 1750.00 1702.00 43583.00


Great work! You're done.